In [ ]:
import cv2
import numpy as np
from sort import Sort
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Initialize SORT tracker
tracker = Sort()
cap = cv2.VideoCapture("../../00.Data/08.YOLO/people_walk.mp4")

previous_positions = {}

def detect_objects(frame):
    boxes = model(frame)[0].boxes.xyxy.cpu().numpy()
    confidences = model(frame)[0].boxes.conf.cpu().numpy()
    detections = np.concatenate((boxes, confidences[:, np.newaxis]), axis=1)
    return detections

def calculate_speed_and_direction(prev_pos, current_pos, time_interval):
    dx = current_pos[0] - prev_pos[0]
    dy = current_pos[1] - prev_pos[1]
    distance = np.sqrt(dx**2 + dy**2)
    speed = distance / time_interval
    direction = np.arctan2(dy, dx) * 180 / np.pi  # in degrees
    return speed, direction

def draw_arrow_and_trajectory(frame, obj_id, center_x, center_y, prev_pos):
    positions_list = previous_positions.get(obj_id, None)

    if positions_list is None:
        positions_list = []
        previous_positions[obj_id] = positions_list

    if prev_pos:
        previous_positions[obj_id].append(prev_pos)
        for i in range(1, len(previous_positions[obj_id])):
            cv2.line(frame, previous_positions[obj_id][i-1], previous_positions[obj_id][i], (0, 255, 0), thickness=2)
        cv2.arrowedLine(frame, (int(prev_pos[0]), int(prev_pos[1])), (int(center_x), int(center_y)), (255, 0, 0), thickness=2, tipLength=0.1)

frame_rate = cap.get(cv2.CAP_PROP_FPS)
time_interval = 1.0 / frame_rate

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Ensure frame is in RGB format as expected by the model
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    detections = detect_objects(frame_rgb)
    dets = np.array([[x1, y1, x2, y2, conf] for x1, y1, x2, y2, conf in detections])
    tracked_objects = tracker.update(dets)

    for obj in tracked_objects:
        x1, y1, x2, y2, obj_id = obj[:5]
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        current_pos = (center_x, center_y)

        prev_pos = previous_positions.get(obj_id, None)

        if prev_pos:
            speed, direction = calculate_speed_and_direction(prev_pos, current_pos, time_interval)
            cv2.putText(frame, f'ID: {int(obj_id)} Speed: {speed:.2f} Direction: {direction:.2f}',
                        (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        #draw_arrow_and_trajectory(frame, obj_id, center_x, center_y, prev_pos)

        previous_positions[obj_id] = current_pos

        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (150, 50, 50), 2)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()